<a href="https://colab.research.google.com/github/agu3ro/PyTorch-DL-Linkedin-Learning/blob/main/05_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.utils.data import DataLoader, random_split
import numpy as np
import pandas as pd
from torchvision import datasets, transforms
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# Validation and testing


# Define NN, init and forward functions

class Model(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=torch.nn.Conv2d(3,6,3,1)
    self.conv2=torch.nn.Conv2d(6,16,3,1)
    self.fc1 = torch.nn.Linear(16*6*6,128)
    self.fc2 = torch.nn.Linear(128,84)
    self.fc3 = torch.nn.Linear(84,10)

  def forward(self,x):
    x = torch.nn.functional.relu(self.conv1(x))
    x = torch.nn.functional.max_pool2d(x,2,2)
    x = torch.nn.functional.relu(self.conv2(x))
    x = torch.nn.functional.max_pool2d(x,2,2)
    x = x.view(-1,16*6*6)
    x = torch.nn.functional.relu(self.fc1(x))
    x = torch.nn.functional.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [3]:
# Instantiate the Model

model = Model()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [4]:
# Load and transform the data

train_data = datasets.CIFAR10(root='./data', download=True, train=True,
                              transform=transforms.Compose([transforms.ToTensor(),
                                                            transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))]))

train_set, val_set = random_split(train_data, [40000,10000])

test_data =  datasets.CIFAR10(root='./data', download=True, train=False,
                              transform=transforms.Compose([transforms.ToTensor(),
                                                            transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))]))
val_loader = DataLoader(val_set, batch_size=16, shuffle=True)

train_loader = DataLoader(train_set, batch_size=4, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transforms.Compose([transforms.ToTensor(),
                                                            transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))]))
test_loader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:01<00:00, 91256656.30it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Set the model to training mode and evaluation mode for validation

for epoch in range(10):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for b, (X_train, y_train) in enumerate(train_loader):
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()  # Set the model to evaluation mode for validation
    validation_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for (X_test, y_test) in test_loader:
            y_prede = model(X_test)
            loss = criterion(y_prede, y_test)
            validation_loss += loss.item()
            predicted = torch.max(y_prede.data, 1)[1]
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()

    print(f'Epoch {epoch + 1}, Training Loss: {running_loss / len(train_loader)}, Validation Loss: {validation_loss / len(val_loader)}, Validation Accuracy: {100 * correct / total}%')

Epoch 1, Training Loss: 2.1564428816497325, Validation Loss: 7.743628835296631, Validation Accuracy: 31.55%
Epoch 2, Training Loss: 1.8033025344014169, Validation Loss: 6.610520262336731, Validation Accuracy: 39.99%
Epoch 3, Training Loss: 1.5837894797086716, Validation Loss: 5.961226272439957, Validation Accuracy: 46.11%
Epoch 4, Training Loss: 1.4614331464439632, Validation Loss: 5.683153637337685, Validation Accuracy: 48.81%
Epoch 5, Training Loss: 1.377677580024302, Validation Loss: 5.471713657689095, Validation Accuracy: 50.39%
Epoch 6, Training Loss: 1.3039605216667056, Validation Loss: 5.152229801154137, Validation Accuracy: 54.27%
Epoch 7, Training Loss: 1.2408536300227047, Validation Loss: 4.993009242486954, Validation Accuracy: 56.0%
Epoch 8, Training Loss: 1.1842465287715196, Validation Loss: 4.878485513985157, Validation Accuracy: 56.86%
Epoch 9, Training Loss: 1.134928352862969, Validation Loss: 4.693570006871224, Validation Accuracy: 58.73%
Epoch 10, Training Loss: 1.0936